In [1]:
MARKET = 'SNP'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = DATAPATH.joinpath('temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [6]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [7]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_ohlcs.pkl', 'df_opt_prices.pkl',
       'df_symlots.pkl', 'df_und_margins.pkl', 'df_unds.pkl', 'dfrq.pkl',
       'qopt_rejects.pkl', 'qopts.pkl'], dtype='<U18')

,conId
0,0


# Making margins robust
* [x] make wifAsync work for one contract
* [x] find optimum settings for margins in executeAsync (checked in dfrq!)

## 1. One contract wifAsync

In [ ]:
# imports
from engine import pre_process
from support import get_dte

In [ ]:
# build a set of contracts to test
df = util.df(qopts.to_list()).iloc[:, 1:6].rename(columns={'lastTradeDateOrContractMonth': 'expiry'})

df['contract'] = qopts
df['order'] = [MarketOrder('SELL', 1)]*len(df)
df['dte'] = df.expiry.apply(get_dte)
df = df[df.dte>5].reset_index(drop=True)

In [ ]:
# set the inputs
ct = df.sample(1).contract.iloc[0]
o = df[df.conId == ct.conId].order.iloc[0]

co = (ct, o)

kwargs = {'FILL_DELAY': 0.25}

In [ ]:
async def margin(ib: IB, co, **kwargs) -> pd.DataFrame:
    
    try:
        FILL_DELAY = kwargs['FILL_DELAY']
    except KeyError as ke:
        print(f"\nWarning: No FILL_DELAY supplied!. 1.5 second default is taken\n")
        FILL_DELAY = 1.5
    
    empty_df = pd.DataFrame({
                "initMarginChange": np.nan,
                "maxCommission": np.nan,
                "commission": np.nan,
                },
                index=range(1))
    
    try:
        ct, o = pre_process(co)
    except ValueError as ve:
        print(f"\nError: {co} co supplied is incorrect! It should be a tuple(ct, o)\n")
        df = empty_df
        
    async def wifAsync(ct, o):
        wif = ib.whatIfOrderAsync(ct, o)
        await asyncio.sleep(FILL_DELAY)
        return wif
    
    wif = await wifAsync(ct, o)
    
    if wif.done():
        
        res = wif.result()
        
        try:
        
            df = util.df([res])[["initMarginChange", "maxCommission", "commission"]]
            
        except TypeError as e:
            
            print(f"\nError: Unknown type of contract: {c}, order: {o}" + 
                  f" \n...in margin wif: {wif} !\n" + 
                  f"   \n..... giving error{e}")
            
            df = empty_df
            
        except IndexError as e:
            
            print(f"\nError: Index error for contract: {c}, order: {o}" + 
                  f" \n...in margin wif: {wif} \n" + 
                  f"   \n..... giving error{e}")
            
            df = empty_df
    
    else:
        
        print(f"\nError: wif could not complete for contract: {ct.localSymbol}"+
              f"\nTry by increasing FILL_DELAY from > {FILL_DELAY} secs\n")
        
        df = empty_df

    # post-processing df
    df = df.assign(secType = ct.secType, conId=ct.conId, localSymbol=ct.localSymbol, symbol=ct.symbol)
    df = df.assign(
            comm=df[["commission", "maxCommission"]].min(axis=1),
            margin=df.initMarginChange.astype("float"))

    # Correct unrealistic margin and commission
    df = df.assign(margin=np.where(df.margin > 1e7, np.nan, df.margin),
        comm=np.where(df.comm > 1e7, np.nan, df.comm))
    
    df = df[['conId', 'symbol', 'secType', 'localSymbol', 'margin', 'comm']]
    
    return df


In [ ]:
%%time
with IB().connect(HOST, PORT, CID) as ib:
    dfm = ib.run(margin(ib, co, **kwargs))

In [ ]:
dfm